In [1]:
import pandas as pd

In [2]:
import altair as alt

In [3]:
import re

In [49]:
import bokeh.plotting as bp


In [5]:
from bokeh.io import output_notebook

In [62]:
from bokeh.models import ColumnDataSource

In [6]:
output_notebook()

Loading BokehJS ...

In [7]:
pd.set_option('display.expand_frame_repr', False)

In [8]:
df = pd.read_csv("/Users/josemartinez/Desktop/cleaned_critical_incidents.csv")

In [9]:
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [10]:
df.columns.values

array(['facility_name', 'investigation_name', 'start_date',
       'incident_date', 'reported_date', 'finalized_date', 'summary',
       'conclusion', 'file_path', 'start_date_year', 'incident_date_year',
       'reported_date_year', 'finalized_date_year'], dtype=object)

In [11]:
df.summary.count()

273

In [12]:
df.investigation_name.str.contains('sexual',case=False,regex=False).sum()

36

In [13]:
(36/273)*100

13.186813186813188

In [14]:
df.investigation_name.isna().sum()

11

In [15]:
sa=df[['investigation_name','start_date_year']]

In [16]:
sa.shape

(273, 2)

In [17]:
sa1 = sa[(sa['start_date_year']== 2020)].investigation_name.str.contains('sexual',case=False,regex=False).sum()

In [18]:
print(f'''
     Investigations Related to Sexual Misconduct or Abuse in 2020 = {sa[(sa['start_date_year']== 2020)].investigation_name.str.contains('sexual',case=False,regex=False).sum()}
     Investigations Related to Sexual Misconduct or Abuse in 2019 = {sa[(sa['start_date_year']== 2019)].investigation_name.str.contains('sexual',case=False,regex=False).sum()}
     Investigations Related to Sexual Misconduct or Abuse in 2018 = {sa[(sa['start_date_year']== 2018)].investigation_name.str.contains('sexual',case=False,regex=False).sum()}
     Investigations Related to Sexual Misconduct or Abuse in 2017= {sa[(sa['start_date_year']== 2017)].investigation_name.str.contains('sexual',case=False,regex=False).sum()}''')


     Investigations Related to Sexual Misconduct or Abuse in 2020 = 14
     Investigations Related to Sexual Misconduct or Abuse in 2019 = 19
     Investigations Related to Sexual Misconduct or Abuse in 2018 = 1
     Investigations Related to Sexual Misconduct or Abuse in 2017= 0


In [19]:
df1 = df.facility_name.value_counts().rename_axis('Facilities').reset_index(name='Incidents')

In [20]:
df1.shape

(93, 2)

In [21]:
data = df1[:5]

In [22]:
data1 = data.copy()

In [23]:
data1['Facilities'] = data1['Facilities'].str.replace('__critical_conditions', '').str.replace('_',' ').str.title()

In [72]:
print(data1)

                                          Facilities  Incidents
0  Provo Canyon School Provo Canyon Springville  ...         15
1  Provo Canyon School Provo Canyon School   Crit...         14
2  Three Points Center Three Points Center   Crit...         10
3  Youth Health Associates Draper Ranch Yha Drape...         10
4  Cinnamon Hills Youth Crisis Center Cinnamon Hi...          9


In [25]:
alt.Chart(data1).mark_bar().encode(
x='Facilities',
y='Incidents:Q',
color=alt.value("#FF9F8A")).properties(
    width=1000,
    height=500
)

alt.Chart(...)

In [81]:
source = ColumnDataSource(data1)

In [108]:
x = data1['Facilities']

In [109]:
y = data1['Incidents']

In [110]:
p = figure(
        title='Critical Incidents per Facility',
        x_axis_label='Facilities',
        y_axis_label='Incidents')

In [116]:
p.vbar(x, y)

GlyphRenderer(id='4426', ...)

In [118]:
show(p)

In [119]:
data.columns

Index(['Facilities', 'Incidents'], dtype='object')

In [28]:
dfstart=df['start_date_year']

In [29]:
dfstart.value_counts()

2019.0    126
2020.0    112
2018.0     21
2017.0      9
Name: start_date_year, dtype: int64

In [36]:
df.reported_date_year.isna().sum()

70

In [37]:
df.start_date_year.isna().sum()

5

In [38]:
df.finalized_date_year.isna().sum()

3

In [48]:
print(f'''
     Reported Date Year NA = {df.reported_date_year.isna().sum()}
     Start Date Year NA = {df.start_date_year.isna().sum()}
     Finalized Date Year NA = {df.finalized_date_year.isna().sum()}
     Incident Date Year NA = {df.incident_date_year.isna().sum()}''')


     Reported Date Year NA = 70
     Start Date Year NA = 5
     Finalized Date Year NA = 3
     Incident Date Year NA = 72
